In [1]:
import Bio
import Mikado
import pyfaidx

In [2]:
! grep TraesCS1A01G000100.1 ref.bed12 target.bed12

ref.bed12:TraesCS1A01G000100.1	0	1308	ID=TraesCS1A01G000100.1;coding=True;phase=0	0	+	379	640	0	3	250,424,634	0,250,674
target.bed12:TraesCS1A01G000100.1.mrna1	0	1293	ID=TraesCS1A01G000100.1.mrna1;coding=True;phase=0	0	+	379	682	0	1	1293	0


In [3]:
ref_bed = Mikado.parsers.bed12.BED12("TraesCS1A01G000100.1	0	1308	ID=TraesCS1A01G000100.1;coding=True;phase=0	0	+	379	640	0	3	250,424,634	0,250,674")
tar_bed = Mikado.parsers.bed12.BED12("TraesCS1A01G000100.1.mrna1	0	1293	ID=TraesCS1A01G000100.1.mrna1;coding=True;phase=0	0	+	379	682	0	1	1293	0")

In [4]:
cdnas = dict()
cdnas["ref"] = pyfaidx.Fasta("ref.cdnas.fasta")
cdnas["target"] = pyfaidx.Fasta("target.cdnas.fasta")

In [5]:
tar_cdna = str(cdnas["target"][tar_bed.chrom])
ref_cdna = str(cdnas["ref"][ref_bed.chrom])

In [6]:
import parasail

In [9]:
ref_cds = Bio.Seq.Seq(ref_cdna[ref_bed.thick_start - 1:ref_bed.thick_end])
ref_pep = Bio.Seq.Seq(ref_cdna[ref_bed.thick_start - 1:ref_bed.thick_end]).translate()

In [18]:
str(tar_cdna)

'GAGCCCGGTCCTCAGAGCCAATCCTTTTCCCGAAGTTACGGATCCGTTTTGCCGACTTCCCTTGCCTACATTGTTCCATTGGCCAGAGGCTGTTCGCCTTGGAGACCTGATGCGGTTATGAGTACGACCGGGCGTGAACGGTACTCGGTCGTCCGGATTTTCATGGGCCGCCGGGGGCGCACCGGACATCGCGCGACGTGCGGTGCTCTTTCGGCCACTGGACCCTACCTCCGGCTGAACGGTTTCCAAGGTTGGCAGGCCGTTAAGCAGAAAAGATAACTCTTCCCGAGGCCCCCGCCGGCGTCTTCGGACTTCCTAACGTCGCCGTCAACCGCCACATCCCGGCTCGGTAAGTCTTAACCTGATTCCCTTTCGGGGTATGCGCGTGATCGCGCTATCTGCCGGGGTTACCCCGTCCCTTAGGATCGGCTTACCCATGTGCAAGTGCCGTTCACATGGAACCTTTCTAATCTTCGGCCTTCAAAGTTCTCATTTGAATATTTGCTACTACCACCAGCCGCTCCACCCGGGCTCGCGCCCCGGGTTTTGCAGCGGCCGCCGCTCCCTCCTACTCATCGGGGCATGGCGCTCGCCCAGATGGCCGGGTGTGGGTCACGCGCTTCAGCGCCATCCATTTTCGGGGCTAGTTGATTCGGCAGGTGAGTTGTTACACACTCCTTAGCGGATTTCGACTTCCATGACCACCGTCCTGCTGTCTTAATCGACCAACACCCTTTGTGTGTTCTAGGTTAGCGCGCAGTTGGGCACCGTAACCCGGCTTCCGGTTCATCCCGCATCGCCAGTTCTGCTTACCAAAAATGGCCCACTAGGAGCACCCGATTCCGTGGCACGGCTCACCGAATCAGTCGCGCCATCCTACCTATTTAAAGTTTGAGAATAGGTCGAGGACGTTGCGTCCCCGATGCCTCTAATCATTAGCTTTACCTGATAGAACTCGTAATGGGCGCCAGCTATCCTGAGGGAAACTTCGGAGGGAAC

In [12]:
res = parasail.sg_trace_scan_32(str(ref_cds), tar_cdna, 11, 1, parasail.blosum62)

In [14]:
res.cigar.decode

b'379D89=2X46=15I7=2X37=1X29=1X21=1X7=1X2=668D'

In [15]:
res_cdna = parasail.sg_trace_scan_32(str(ref_cdna), tar_cdna, 11, 1, parasail.blosum62)

In [16]:
res_cdna.cigar.decode

b'6=1X11=1X70=1X5=1X2=1X34=1X16=1X37=1X1=1X19=1X2=1X26=1X7=1X5=1X42=1X8=1X21=1X19=1X1=1X2=1X8=1X15=1X89=2X46=15I7=2X37=1X29=1X21=1X7=1X670='

In [23]:
import textwrap

with open("/tmp/query.fa", "wt") as out:
    print(">query", file=out)
    print(*textwrap.wrap(str(ref_cds)), file=out, sep="\n")
    
with open("/tmp/db.fa", "wt") as out:
    print(">sub", file=out)
    print(*textwrap.wrap(tar_cdna), file=out, sep="\n")

In [32]:
ref_pep = str(ref_pep)
print(ref_pep)

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLLFGLQSSHLNICYYHQDLPRRPLCPGSRPGFCSGRRALLLIGAWRSSRWPGVGRAL*


In [35]:
frames = dict()
for num in range(3):
    frames[num] = str(Bio.Seq.Seq(str(tar_cdna[num:])).translate())

In [128]:
frame_res = dict()
for num in range(3):
    frame_res[num] = parasail.sg_trace_scan_32(ref_pep, frames[num], 3, 1, parasail.blosum85)
    
    

In [129]:
for num in [1]:
    print(num, frame_res[num].cigar.decode, frame_res[num].score)
    print()

1 b'126D30=1X15=5I2=1X12=1X9=1X6=1X2=1I223D' 447



In [92]:
print(frames[1][126:126+46] + "-" * 5 + frames[1][126+46:126+46+2])
print("|" * 30 + "X" + "|" * 15 + " " * 5 + "|" * 2 + " " + "X" + "|")
print(ref_pep[:53] + "-" + ref_pep[53:53+2])

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLIFGLQSSHLNICYYHQ-----PL
||||||||||||||||||||||||||||||X|||||||||||||||     || X|
MRVIALSAGVTPSLRIGLPMCKCRSHGTFLLFGLQSSHLNICYYHQDLPRRPL-CP


In [96]:
values = [(30, "="), (1, "X"), (15, "="), (5, "I"), (2, "="), (1, "X"), (12, "="), (1, "X"), (9, "="), (1, "X"), (6, "="), (1, "X"), (2, "=")]

In [130]:
ref_str = ""
middle = ""
tar_str = ""

tar_pep = frames[1][126:]
ref_pos, tar_pos = 0, 0


for l, code in values:
    if code in ("=", "X"):
        ref_str += ref_pep[ref_pos:ref_pos + l]
        if code == "=":
            char = "|"
        else:
            char = "X"
        middle += char * l
        tar_str += tar_pep[tar_pos:tar_pos + l]
        tar_pos +=l
        ref_pos +=l
    elif code == "I":
        ref_str += ref_pep[ref_pos:ref_pos + l]
        tar_str += "-" * l
        middle += " " * l
        ref_pos += l
    elif code == "D":
        tar_str += tar_pep[tar_pos: tar_pos + l]
        ref_str += "-" * l
        middle += " " * l
        tar_pos += l
    

In [131]:
print(ref_str)
print(middle)
print(tar_str)

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLLFGLQSSHLNICYYHQDLPRRPLCPGSRPGFCSGRRALLLIGAWRSSRWPGVGRAL
||||||||||||||||||||||||||||||X|||||||||||||||     ||X||||||||||||X|||||||||X||||||X||
MRVIALSAGVTPSLRIGLPMCKCRSHGTFLIFGLQSSHLNICYYHQ-----PLHPGSRPGFCSGRRSLLLIGAWRSPRWPGVGHAL


In [101]:
frames[1][126:]

'MRVIALSAGVTPSLRIGLPMCKCRSHGTFLIFGLQSSHLNICYYHQPLHPGSRPGFCSGRRSLLLIGAWRSPRWPGVGHALQRHPFSGLVDSAGELLHTP*RISTSMTTVLLS*STNTLCVF*VSAQLGTVTRLPVHPASPVLLTKNGPLGAPDSVARLTESVAPSYLFKV*E*VEDVASPMPLIISFT**NS*WAPAILRETSEGTSY*MV*LVFRPYNQVRRTICTLVLLRASTRVSSGFALLRHSSPSFGSRQACSNSNPSQKIRVSQRCGP*GPPARQLPCASQVSEPVDSHACQTPWSV'

In [112]:
print(len(tar_str))

86


In [107]:
print(126*3)

378


In [119]:
print(tar_str +   str(Bio.Seq.Seq(str(tar_cdna[379+81 * 3:])).translate(to_stop=True)))

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLIFGLQSSHLNICYYHQ-----PLHPGSRPGFCSGRRSLLLIGAWRSPRWPGVGHALQRHPFSGLVDSAGELLHTP


In [116]:
print(ref_pep)

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLLFGLQSSHLNICYYHQDLPRRPLCPGSRPGFCSGRRALLLIGAWRSSRWPGVGRAL*


In [120]:
print(ref_pep)
print(middle)
print(tar_str +   str(Bio.Seq.Seq(str(tar_cdna[379+81 * 3:])).translate(to_stop=True)))

MRVIALSAGVTPSLRIGLPMCKCRSHGTFLLFGLQSSHLNICYYHQDLPRRPLCPGSRPGFCSGRRALLLIGAWRSSRWPGVGRAL*
||||||||||||||||||||||||||||||X|||||||||||||||     ||X||||||||||||X|||||||||X||||||X||
MRVIALSAGVTPSLRIGLPMCKCRSHGTFLIFGLQSSHLNICYYHQ-----PLHPGSRPGFCSGRRSLLLIGAWRSPRWPGVGHALQRHPFSGLVDSAGELLHTP


In [182]:
! grep TraesCS1A01G000100.1.mrna1 target.gff3

Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	mRNA	1310	2602	.	+	.	ID=TraesCS1A01G000100.1.mrna1;Name=TraesCS1A01G000100.1;Parent=TraesCS1A01G000100.1.path1;coverage=100.0;identity=96.6;matches=1263;mismatches=30;indels=15;unknowns=0
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	exon	1310	2602	96	+	.	ID=TraesCS1A01G000100.1.mrna1.exon1;Name=TraesCS1A01G000100.1;Parent=TraesCS1A01G000100.1.mrna1;Target=TraesCS1A01G000100.1 1308 1 .
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	CDS	1689	1991	92	+	0	ID=TraesCS1A01G000100.1.mrna1.cds1;Name=TraesCS1A01G000100.1;Parent=TraesCS1A01G000100.1.mrna1;Target=TraesCS1A01G000100.1 697 380 .


In [184]:
tar = Mikado.transcripts.Transcript(Mikado.parsers.GFF.GffLine("Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	mRNA	1310	2602	.	+	.	ID=TraesCS1A01G000100.1.mrna1;Name=TraesCS1A01G000100.1;Parent=TraesCS1A01G000100.1.path1;coverage=100.0;identity=96.6;matches=1263;mismatches=30;indels=15;unknowns=0"))
tar.add_exons([(1310, 2602)])
tar.add_exons([(1689, 1991)], features="CDS")
tar.finalize()


In [192]:
from eiliftover.util import transfer_utilities as transfer
from Bio import Seq

def transfer_by_alignment(ref_pep, target_cdna, target_bed):

    frames = dict()
    # Get the three-frame translation
    for frame in range(3):
        frames[frame] = str(Seq.Seq(target_cdna[frame:]).translate(to_stop=False))

    # This will get the best match in the 3-frame translation
    aln, best_frame, score, best_cigar = None, None, float("-inf"), None
    for frame in frames:
        res, cigar = transfer.get_and_prepare_cigar(ref_pep, frames[frame], open=3, extend=1, matrix=parasail.blosum85)
        if res.score > score:
            aln, best_frame, score, best_cigar = res, frame, res.score, cigar

    # Now it is time to try to transfer it ... Ignore any deletions at the beginning
    cig_start = 0
    translation_start = 0
    
    while not transfer.op_consumes[best_cigar[cig_start][1]][0]:
        translation_start += best_cigar[cig_start][0]
        cig_start += 1
        
    # print(cig_start, best_cigar[cig_start:])
    # This is 0-based; we have to add 1 because we start 1 base after the gap at the beginning
    if translation_start > 0:
        translation_start = 3 * translation_start + 1
    
    translated = Seq.Seq(target_cdna[translation_start:]).translate(to_stop=True)
    # Logic to handle when the CDS is broken
    # This is 1-based, so we have to add 1 to 
    target_bed.thick_start = translation_start + 1
    end = target_bed.thick_start + len(translated) * 3 - 1
    
    if translated[0] != ref_pep[0]:
        if translation_start in (0, 1, 2):
            target_bed.phase = translation_start
        else:
            target_bed.coding = False
            return target_bed
    
    # Now check whether we can add the stop codon
    if end + 3 < len(target_cdna):
        end += 3
    else: # Here we have to presume that it is open.
        end = len(target_cdna)
    
    # print(translation_start * 3, translated)
    target_bed.thick_end = end
    target_bed.coding = True
    return target_bed

tbed = transfer_by_alignment(ref_pep, tar_cdna, tar_bed)
tbed.transcriptomic = True
print(tbed)
print(tar_bed)
print(tbed.coding, tbed.invalid, tbed.invalid_reason)

ntar = tar.copy()
ntar.strip_cds()
logger = Mikado.utilities.log_utils.create_default_logger("test")
ntar.logger = logger
ntar.logger.setLevel("DEBUG")
ntar.unfinalize()
ntar.load_orfs([tbed])
print(ntar.cdna_length)
print(ntar.combined_cds_length)

ntar.logger.setLevel("WARNING")
print(ntar.format("gff3"))


2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,347 - test - finalizing.py:57 - DEBUG - __basic_final_checks - MainProcess - Converting to tuples
2018-04-25 15:47:38,369 - test - finalizing.py:117 - DEBUG - _check_cdna_vs_utr - MainProcess - Checking the cDNA for TraesCS1A01G000100.1.mrna1
2018-04-25 15:47:38,369 - test - finalizing.py:117 - DEBUG - _check_cdna_vs_utr - MainProcess - Checking the cDNA for TraesCS1A01G000100.1.mrna1
20

TraesCS1A01G000100.1.mrna1	0	1293	TraesCS1A01G000100.1.mrna1	0	+	379	682	0	1	1293	0
TraesCS1A01G000100.1.mrna1	0	1293	TraesCS1A01G000100.1.mrna1	0	+	379	682	0	1	1293	0
True False 


2018-04-25 15:47:38,541 - test - clique_methods.py:98 - DEBUG - find_cliques - MainProcess - Creating cliques for test
2018-04-25 15:47:38,541 - test - clique_methods.py:98 - DEBUG - find_cliques - MainProcess - Creating cliques for test
2018-04-25 15:47:38,541 - test - clique_methods.py:98 - DEBUG - find_cliques - MainProcess - Creating cliques for test
2018-04-25 15:47:38,541 - test - clique_methods.py:98 - DEBUG - find_cliques - MainProcess - Creating cliques for test
2018-04-25 15:47:38,541 - test - clique_methods.py:98 - DEBUG - find_cliques - MainProcess - Creating cliques for test
2018-04-25 15:47:38,561 - test - clique_methods.py:100 - DEBUG - find_cliques - MainProcess - Created 1 cliques for test
2018-04-25 15:47:38,561 - test - clique_methods.py:100 - DEBUG - find_cliques - MainProcess - Created 1 cliques for test
2018-04-25 15:47:38,561 - test - clique_methods.py:100 - DEBUG - find_cliques - MainProcess - Created 1 cliques for test
2018-04-25 15:47:38,561 - test - clique_me

1293
303
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	mRNA	1310	2602	.	+	.	ID=TraesCS1A01G000100.1.mrna1;Parent=TraesCS1A01G000100.1.path1;Name=TraesCS1A01G000100.1.mrna1;coverage=100.0;identity=96.6;indels=15;matches=1263;mismatches=30;unknowns=0
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	exon	1310	2602	.	+	.	ID=TraesCS1A01G000100.1.mrna1.exon1;Parent=TraesCS1A01G000100.1.mrna1
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	five_prime_UTR	1310	1688	.	+	.	ID=TraesCS1A01G000100.1.mrna1.five_prime_UTR1;Parent=TraesCS1A01G000100.1.mrna1
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	CDS	1689	1991	.	+	0	ID=TraesCS1A01G000100.1.mrna1.CDS1;Parent=TraesCS1A01G000100.1.mrna1
Triticum_aestivum_CS42_TGACv1_scaffold_021338_1AS	Triticum_aestivum_TGACv1_all	three_prime_UTR	1992	2602	.	+	.	ID=TraesCS1A01G000100.1.mrna1.three_prime_UTR1;Parent=TraesCS1A01G000100.1.mrna

In [169]:
! grep TraesCS1A01G000100LC.1 *bed12

ref.bed12:TraesCS1A01G000100LC.1	0	2740	ID=TraesCS1A01G000100LC.1;coding=True;phase=0	0	+	1122	1443	0	3	417,432,1891	0,417,849
target.bed12:TraesCS1A01G000100LC.1.mrna1	0	2731	ID=TraesCS1A01G000100LC.1.mrna1;coding=True;phase=0	0	+	1222	1519	0	2	1640,1091	0,1640


In [171]:
nref = Mikado.parsers.bed12.BED12("TraesCS1A01G000100LC.1	0	2740	ID=TraesCS1A01G000100LC.1;coding=True;phase=0	0	+	1122	1443	0	3	417,432,1891	0,417,849")
ntar = Mikado.parsers.bed12.BED12("TraesCS1A01G000100LC.1.mrna1	0	2731	ID=TraesCS1A01G000100LC.1.mrna1;coding=True;phase=0	0	+	1222	1519	0	2	1640,1091	0,1640")

In [176]:
print(str(Seq.Seq(str(cdnas["ref"][nref.chrom][nref.thick_start - 1:nref.thick_end])).translate()))
print(str(Seq.Seq(str(cdnas["target"][ntar.chrom][ntar.thick_start - 1:ntar.thick_end])).translate()))

MSGAPPAAHENPDDRVPFTPGRTHNRIRSPRRTASGQWNNVGKGSRQNGSVTSGKGLALRTGLGGPGPEPVGCRRIARAAHVARAGRRVPAGGRTGNHPFGGFPRA*
MEQCRQGKSAKRIRNFGKTIGSEDWARGSLPRTRRLSADCSSCSRGESGSSRAGRGTDRESPLRGLSPSMKQSTQNWYGQGESDCLIKTKHCDGPRGC*


In [179]:
tar_cdna

'GAGCCCGGTCCTCAGAGCCAATCCTTTTCCCGAAGTTACGGATCCGTTTTGCCGACTTCCCTTGCCTACATTGTTCCATTGGCCAGAGGCTGTTCGCCTTGGAGACCTGATGCGGTTATGAGTACGACCGGGCGTGAACGGTACTCGGTCGTCCGGATTTTCATGGGCCGCCGGGGGCGCACCGGACATCGCGCGACGTGCGGTGCTCTTTCGGCCACTGGACCCTACCTCCGGCTGAACGGTTTCCAAGGTTGGCAGGCCGTTAAGCAGAAAAGATAACTCTTCCCGAGGCCCCCGCCGGCGTCTTCGGACTTCCTAACGTCGCCGTCAACCGCCACATCCCGGCTCGGTAAGTCTTAACCTGATTCCCTTTCGGGGTATGCGCGTGATCGCGCTATCTGCCGGGGTTACCCCGTCCCTTAGGATCGGCTTACCCATGTGCAAGTGCCGTTCACATGGAACCTTTCTAATCTTCGGCCTTCAAAGTTCTCATTTGAATATTTGCTACTACCACCAGCCGCTCCACCCGGGCTCGCGCCCCGGGTTTTGCAGCGGCCGCCGCTCCCTCCTACTCATCGGGGCATGGCGCTCGCCCAGATGGCCGGGTGTGGGTCACGCGCTTCAGCGCCATCCATTTTCGGGGCTAGTTGATTCGGCAGGTGAGTTGTTACACACTCCTTAGCGGATTTCGACTTCCATGACCACCGTCCTGCTGTCTTAATCGACCAACACCCTTTGTGTGTTCTAGGTTAGCGCGCAGTTGGGCACCGTAACCCGGCTTCCGGTTCATCCCGCATCGCCAGTTCTGCTTACCAAAAATGGCCCACTAGGAGCACCCGATTCCGTGGCACGGCTCACCGAATCAGTCGCGCCATCCTACCTATTTAAAGTTTGAGAATAGGTCGAGGACGTTGCGTCCCCGATGCCTCTAATCATTAGCTTTACCTGATAGAACTCGTAATGGGCGCCAGCTATCCTGAGGGAAACTTCGGAGGGAAC